# CycleGAN batch inference
This notebooks provides inference functionality.

In [ ]:
#default_exp inference.cyclegan

In [ ]:
#export 
from upit.models.cyclegan import *
from upit.train.cyclegan import *
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
import glob
import tqdm
import os
import PIL

## Batch inference functionality

In [ ]:
#export
class FolderDataset(Dataset):
    import PIL
    def __init__(self, path,transforms=None):
        self.files = glob.glob(path+'/*')
        self.totensor = torchvision.transforms.ToTensor()
        if transforms:
            self.transform = torchvision.transforms.Compose(transforms)
        else:
            self.transform = lambda x: x
    
    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        image = PIL.Image.open(self.files[idx % len(self.files)])
        image = self.totensor(image)
        image = self.transform(image)
        return self.files[idx], image

def load_dataset(test_path,bs=4,num_workers=4):
        dataset = FolderDataset(
            path=test_path,
            transforms=[torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
        ) 
        loader = torch.utils.data.DataLoader(
            dataset,
            batch_size=bs,
            num_workers=num_workers,
            shuffle=True
        )
        return loader

In [ ]:
#export
def get_preds_cyclegan(learn,test_path,pred_path,bs=4,num_workers=4,suffix='png'):
    
    assert os.path.exists(test_path)
    
    if not os.path.exists(pred_path):
        os.mkdir(pred_path)
        
    model = learn.model.G_A
    
    test_dl = load_dataset(test_path,bs,num_workers)
    
    for i, xb in tqdm.tqdm(enumerate(test_dl),total=len(test_dl)):
        fn, im = xb
        preds = (learn.model.G_B(im.cuda())/2 + 0.5)
        for i in range(len(fn)):
            new_fn = os.path.join(pred_path,'.'.join([os.path.basename(fn[i]).split('.')[0]+'_fakeB',suffix]))                  
            torchvision.utils.save_image(preds[i],new_fn)

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 01_models.cyclegan.ipynb.
Converted 01b_models.junyanz.ipynb.
Converted 02_data.unpaired.ipynb.
Converted 03_train.cyclegan.ipynb.
Converted 04_inference.cyclegan.ipynb.
Converted index.ipynb.
